SET-COVERING PROBLEM

The set covering problem is a significant NP-hard problem in combinatorial optimization. Given a collection of elements, the set covering problem aims 
to find the minimum number of sets that incorporate (cover) all of these elements. In the set covering problem, two sets are given: a set {U} of elements 
and a set {S} of subsets of the set {U}. Each subset in {S} is associated with a predetermined cost, and the union of all the subsets covers the set 
{U}. This combinatorial problem then concerns finding the optimal number of subsets whose union covers the universal set while minimizing the total cost.

SOLUTION SPACE

What is a state? In this problem a state should be composed of 2 groups: one contains all the taken sets the other one the not taken. Each set contains a
specific number of elements. 
We will need a function that checks for the goal state.
We will use a path searching algorithms.

In [1]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue
from math import ceil

In [2]:
# Since we don't care about the order we decide to use sets instead of arrays to represent the tiles (group of elements)


PROBLEM_SIZE = 50 # Number of elements (so the solution MUST contain them all)
NUM_SETS = 20 # Number of sets/tiles

# random() generates a random float number between 0 and 1
# random() < .x gives me a boolean
# If True it means that the element in that position is covered by the set otherwise it isn't
# EXAMPLE: True, True, False -> Set contains element 0 and 1
# Since I do not want to modify them I can wrap everything in a tuple (that is faster and more memory efficient than an array)
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple("State", "taken not_taken") # From now on when I create a State() the first element will be labeled as taken, the second one as not_taken

In [3]:
# The function that checks if the state is a goal state has to understand if with all the taken sets I'm covering all the elements (PROBLEM_SIZE)
def goal_check(state):
    # The second argument of the reduce function is the initial value (a nparray of all False)
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

# Before going on I check if the problem is solvable in general by putting all sets as taken and see if they cover all the elements
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

# This function counts the number of repetition in a state (used for the variant of the problem)
def repetitions(state):
    if len(state.taken) <= 1:
        return 0
    return sum(filter(lambda x: x > 0, sum(SETS[i] for i in state.taken) - 1))

In [4]:
# PATH SEARCH ALGORITHM

# Define the frontier
# Breadth-first search -> FIFO queue (SimpleQueue)
# Depth-first search -> LIFO queue
# Uniform-cost search (Dijikstra) -> Priority queue

# Depth first

In [5]:
frontier = deque()
# Define the initial state
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

# I want to count the steps it takes to find a solution
counter = 0
# Pop the initial state
current_state = frontier.pop()

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        # Apply the action and create a new state
        # In this case I add a set to the takens and remove it from the not_takens
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.append(new_state)
    current_state = frontier.pop()
    
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles) ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")

Solved in 10 steps (10 tiles) (88 repetitions)
State taken: {10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


# Breadth first

In [6]:
frontier = deque()
# Define the initial state
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

# I want to count the steps it takes to find a solution
counter = 0
# Pop the initial state
current_state = frontier.popleft()

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        # Apply the action and create a new state
        # In this case I add a set to the takens and remove it from the not_takens
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.append(new_state)
    # Breadth first -> FIFO
    current_state = frontier.popleft()
    
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles) ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")

Solved in 130,240 steps (5 tiles) (38 repetitions)
State taken: {0, 2, 19, 8, 14}


# Greedy best-first

In [7]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [8]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        # Apply the action and create a new state
        # In this case I add a set to the takens and remove it from the not_takens
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles) ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")

Solved in 5 steps (5 tiles) (40 repetitions)
State taken: {19, 4, 8, 13, 14}


# A* search

In [9]:
def h(state):
    # Reasoning: My heuristic is based on the best possible set I could take among all
    # if my best set (the one with most elements covered) covers 3 elements and I am missing 6 elements covered
    # then I would need AT LEAST 2 sets (of the largest size) to cover what I'm missing (only if those sets cover the exact elements I'm missing)
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    # This heuristic is tighter: it means that it is admissable but is "less optimistic" than the previous one
    # I'm calculating how many sets I would need in the best scenario, i.e. taking in considerations only the not covered elements
    # (I'm picking up of the biggest set after having removed the element covered by sets taken)
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    # largest set after having removed the already covered elements
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    # Instead of assuming that all the sets I need to cover every element are equal to the largest set, as i did in h2, 
    # now I'm considering all sets by itself; so I sort the sets by the number of elements they cover (from highest to lowest)
    # and I update a counter (taken) till I cover all the elements I'm missing
    # h3 could be a little bit more accurate in those scenario in which the dimensions of the sets (number of elements not yet covered that
    # I actually need) is very variable
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def f(state):
    return len(state.taken) + h(state)

In [10]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles) ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")

Solved in 7,735 steps (5 tiles) (38 repetitions)
State taken: {0, 2, 19, 8, 14}


VARIANT

Instead of looking for the combination that contains the less sets possible to cover all the elements, I will try to search the solution
that minimizes the number of repetitions of each element (less overlap possible)

# Greedy best-first

In [11]:
# What if my objective is to find the sets that permit me to have the less repetition possible?
""" Some example to know if I was doing right
SETS = tuple(np.array([[True, False, False, True, False, False, False, False, False],
                       [True, False, True, False, False, True, True, False, False],
                       [True, True, True, False, False, False, True, False, True],
                       [False, True, False, False, True, False, False, False, True],
                       [False, False, False, True, False, False, False, False, True],
                       [False, False, True, True, True, False, True, True, False],
                       [False, False, False, False, False, True, True, True, False],
                       [True, False, False, True, False, True, False, False, False],
                       [False, True, False, False, False, True, True, True, False],
                       [False, False, False, False, False, True, False, False, False]]))

PROBLEM_SIZE = 9
NUM_SETS = 10
"""

# GREEDY -> repetitions + missing covered elements
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size + repetitions(state)


In [12]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        # Apply the action and create a new state
        # In this case I add a set to the takens and remove it from the not_takens
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")

Solved in 17,270 steps (30 repetitions)
State taken: {19, 4, 6, 10, 14}


# A* search

In [13]:
# In this case my g counts the number of repetition in the taken sets
# the heuristic h MUST take into account the number of already covered elements in the just taken set
# because the more I repeat the less this set should be interesting 
# devo prendere il set che ha il minor numero di already covered elements

def h(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    # best_set is the set with the less repetition possible
    best_set = SETS[np.array(sum(np.logical_and(s, already_covered)) for s in SETS).argmin()]
    best_set_size = sum(best_set)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / best_set_size)
    return optimistic_estimate


# Optimization: I look also at the frequence of each element

FREQUENCE = np.array(sum(s for s in SETS)) / NUM_SETS

def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    # best_set is the set with the less repetition possible that takes the less frequent elements
    best_set = SETS[np.array(sum(np.logical_and(s, already_covered)) + sum(FREQUENCE[np.logical_and(s, np.logical_not(already_covered))]) for s in SETS).argmin()]
    best_set_size = sum(best_set)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / best_set_size)
    return optimistic_estimate

def f_star(state):
    return repetitions(state) + h2(state)

In [14]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f_star(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken | {action}, current_state.not_taken - {action})
        frontier.put((f_star(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({repetitions(current_state)} repetitions)")
print(f"State taken: {current_state.taken}")


# Final Conclusion: It seems that the A* is not working properly since it's giving me the same results of the greedy approach
# Probably the heuristic is not good enough
# The idea i Followed to write the heuristic was taking the size of best set possible and use that one to calculate
# the number of minimal sets I need to cover all the remaining elements (as did for the A* in the first version); but
# the h was taking into account not the largest set but the set with less repetition possible


Solved in 821,729 steps (30 repetitions)
State taken: {19, 4, 6, 10, 14}
